In [1]:
import DBcm

config = "../src/db_sqlite/SwimDB.sqlite3"

In [2]:
print(dir(str))

['__add__', '__class__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mod__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmod__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'capitalize', 'casefold', 'center', 'count', 'encode', 'endswith', 'expandtabs', 'find', 'format', 'format_map', 'index', 'isalnum', 'isalpha', 'isascii', 'isdecimal', 'isdigit', 'isidentifier', 'islower', 'isnumeric', 'isprintable', 'isspace', 'istitle', 'isupper', 'join', 'ljust', 'lower', 'lstrip', 'maketrans', 'partition', 'removeprefix', 'removesuffix', 'replace', 'rfind', 'rindex', 'rjust', 'rpartition', 'rsplit', 'rstrip', 'split', 'splitlines', 'startswith', 'strip', 'swapcase', 'title', 'translate', 'upper', 'zfill']


In [3]:
statement = []
for method in dir(str):
    if not method.startswith("__"):
        statement.append(method)

print(statement)

['capitalize', 'casefold', 'center', 'count', 'encode', 'endswith', 'expandtabs', 'find', 'format', 'format_map', 'index', 'isalnum', 'isalpha', 'isascii', 'isdecimal', 'isdigit', 'isidentifier', 'islower', 'isnumeric', 'isprintable', 'isspace', 'istitle', 'isupper', 'join', 'ljust', 'lower', 'lstrip', 'maketrans', 'partition', 'removeprefix', 'removesuffix', 'replace', 'rfind', 'rindex', 'rjust', 'rpartition', 'rsplit', 'rstrip', 'split', 'splitlines', 'startswith', 'strip', 'swapcase', 'title', 'translate', 'upper', 'zfill']


In [4]:
methods = [method for method in dir(str) if not method.startswith("__")]

In [5]:
SQL_SESSIONS = """
    select distinct ts from times;
"""

SQL_SWIMMERS_BY_SESSION = """
    select distinct swimmers.name, swimmers.age
    from swimmers
    join times on times.swimmer_id = swimmers.id
    where date(times.ts) = ?
    order by swimmers.name; 
"""

SQL_SWIMMERS_EVENTS_BY_SESSION = """
    select distinct events.distance, events.stroke
    from swimmers, events, times
    where 
    swimmers.name = ? and
    swimmers.age = ? and
    date(times.ts) = ? and
    times.swimmer_id = swimmers.id and
    times.event_id = events.id
"""

SQL_CHART_DATA_BY_SWIMMER_EVENT_SESSION = """
    select times.time
    from times, swimmers, events
    where
    swimmers.name = ? and
    swimmers.age = ? and
    events.distance = ? and
    events.stroke = ? and
    date(times.ts) = ? and
    times.swimmer_id = swimmers.id and
    times.event_id = events.id
"""

In [6]:
with DBcm.UseDatabase(config) as db:
    db.execute(SQL_SESSIONS)
    retrieved_times = db.fetchall()
retrieved_times

[('2025-05-12 18:25:07',)]

In [7]:
time_session = retrieved_times[0][0].split(" ")[0]
with DBcm.UseDatabase(config) as db:
    db.execute(SQL_SWIMMERS_BY_SESSION, (time_session,))
    result = db.fetchall()

In [8]:
with DBcm.UseDatabase(config) as db:
    db.execute(SQL_SWIMMERS_EVENTS_BY_SESSION, ("Abi", "10", time_session))
    result = db.fetchall()
result

[('100m', 'Back'),
 ('100m', 'Breast'),
 ('50m', 'Back'),
 ('50m', 'Breast'),
 ('50m', 'Free')]

In [9]:
with DBcm.UseDatabase(config) as db:
    db.execute(SQL_CHART_DATA_BY_SWIMMER_EVENT_SESSION, ("Abi", "10", "100m", "Back", time_session))
    result = db.fetchall()
result

[('1:31.59',), ('1:26.55',), ('1:28.75',), ('1:39.79',), ('1:32.37',)]